In [71]:
import elasticsearch
from elasticsearch_dsl import Search, Q, MultiSearch
import json
import subprocess
import logstash
import requests

In [ ]:
#! docker run -v ~/elk-data:/var/lib/elasticsearch -p 5601:5601 -p 9200:9200 -p 5044:5044 -it sebp/elk:es241_l240_k461

-elastic data is stored in /var/lib/elasticsearch in the container.  
-latest sebp/elk image is failing 


In [70]:
local_machine = {'host':'localhost','port':9200}
es = elasticsearch.Elasticsearch(hosts=[local_machine])


def json_to_num(j):
    for k in j:
        try:
            j[k] = float(j[k])
        except:
            pass
    return j


In [55]:
# quick sanity check if elastic is running
assert 'cluster_name' in requests.get('http://localhost:9200').text

In [56]:
import json
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw2', doc_type='people', id=i, body=json_to_num(json.loads(r.text)))
    i=i+1

KeyboardInterrupt: 

In [ ]:
r = requests.get('http://localhost:9200')
i = 18
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw2', doc_type='people', id=i,  body=json_to_num(json.loads(r.text)))
    i=i+1

In [160]:
res = es.search(index='sw',body={'query':{'match':{"name":'obi'}}})

In [161]:
sorted(res['hits']['hits'], key=lambda x: x['_score'], reverse=True)

[{'_id': '10',
  '_index': 'sw',
  '_score': 0.9581454,
  '_source': {'birth_year': '57BBY',
   'created': '2014-12-10T16:16:29.192000Z',
   'edited': '2014-12-20T21:17:50.325000Z',
   'eye_color': 'blue-gray',
   'films': ['http://swapi.co/api/films/5/',
    'http://swapi.co/api/films/4/',
    'http://swapi.co/api/films/6/',
    'http://swapi.co/api/films/3/',
    'http://swapi.co/api/films/2/',
    'http://swapi.co/api/films/1/'],
   'gender': 'male',
   'hair_color': 'auburn, white',
   'height': '182',
   'homeworld': 'http://swapi.co/api/planets/20/',
   'mass': '77',
   'name': 'Obi-Wan Kenobi',
   'skin_color': 'fair',
   'species': ['http://swapi.co/api/species/1/'],
   'starships': ['http://swapi.co/api/starships/48/',
    'http://swapi.co/api/starships/59/',
    'http://swapi.co/api/starships/64/',
    'http://swapi.co/api/starships/65/',
    'http://swapi.co/api/starships/74/'],
   'url': 'http://swapi.co/api/people/10/',
   'vehicles': ['http://swapi.co/api/vehicles/38/']},

In [50]:
# get current indices
es.indices.get('*').keys()

dict_keys(['.kibana', 'sw'])

In [53]:
es.search(index='sw', body={
    "query" : {
        "match_all" : {}
    }
})


es

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '24',
    '_index': 'sw',
    '_score': 1.0,
    '_source': {'birth_year': '53BBY',
     'created': '2014-12-15T12:53:49.297000Z',
     'edited': '2014-12-20T21:17:50.355000Z',
     'eye_color': 'red',
     'films': ['http://swapi.co/api/films/2/'],
     'gender': 'male',
     'hair_color': 'none',
     'height': '190',
     'homeworld': 'http://swapi.co/api/planets/29/',
     'mass': '113',
     'name': 'Bossk',
     'skin_color': 'green',
     'species': ['http://swapi.co/api/species/7/'],
     'starships': [],
     'url': 'http://swapi.co/api/people/24/',
     'vehicles': []},
    '_type': 'people'},
   {'_id': '25',
    '_index': 'sw',
    '_score': 1.0,
    '_source': {'birth_year': '31BBY',
     'created': '2014-12-15T12:56:32.683000Z',
     'edited': '2014-12-20T21:17:50.357000Z',
     'eye_color': 'brown',
     'films': ['http://swapi.co/api/films/3/', 'http://swapi.co/api/films/2/'],
     'gende

In [73]:
Search(using=es, index='sw').query('match_all').execute().to_dict()

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '24',
    '_index': 'sw',
    '_score': 1.0,
    '_source': {'birth_year': '53BBY',
     'created': '2014-12-15T12:53:49.297000Z',
     'edited': '2014-12-20T21:17:50.355000Z',
     'eye_color': 'red',
     'films': ['http://swapi.co/api/films/2/'],
     'gender': 'male',
     'hair_color': 'none',
     'height': '190',
     'homeworld': 'http://swapi.co/api/planets/29/',
     'mass': '113',
     'name': 'Bossk',
     'skin_color': 'green',
     'species': ['http://swapi.co/api/species/7/'],
     'starships': [],
     'url': 'http://swapi.co/api/people/24/',
     'vehicles': []},
    '_type': 'people'},
   {'_id': '25',
    '_index': 'sw',
    '_score': 1.0,
    '_source': {'birth_year': '31BBY',
     'created': '2014-12-15T12:56:32.683000Z',
     'edited': '2014-12-20T21:17:50.357000Z',
     'eye_color': 'brown',
     'films': ['http://swapi.co/api/films/3/', 'http://swapi.co/api/films/2/'],
     'gende

In [81]:
es.search(index='sw', body={"multi_match": {"fields": ["_all"], "query": "jaba", "fuzziness": "AUTO"}})

RequestError: TransportError(400, 'search_phase_execution_exception', 'failed to parse search source. unknown search element [multi_match]')

In [95]:
# es.search(index='sw', body={
#     "query" : {
#         "multi_match": {"fields": ["_all"], "query": "-wan", "fuzziness": "AUTO"}}
#     }
# )


MultiSearch(using=es, index='sw').match({"fields": ["_all"], "query": "-wan", "fuzziness": "AUTO"})

AttributeError: 'MultiSearch' object has no attribute 'match'

In [110]:
# es.index(index='test', doc_type='bla', body=[{'a':'b'}])


# es.index(index='test2',doc_type='str',body={'a':'b'})


es.search(index='sw', doc_type=None, body={'query':{'match':{"eye_color":'blue'}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '19',
    '_index': 'sw',
    '_score': 2.734601,
    '_source': {'birth_year': 'unknown',
     'created': '2014-12-12T11:16:56.569000Z',
     'edited': '2014-12-20T21:17:50.343000Z',
     'eye_color': 'blue',
     'films': ['http://swapi.co/api/films/1/'],
     'gender': 'male',
     'hair_color': 'brown',
     'height': '180',
     'homeworld': 'http://swapi.co/api/planets/26/',
     'mass': '110',
     'name': 'Jek Tono Porkins',
     'skin_color': 'fair',
     'species': ['http://swapi.co/api/species/1/'],
     'starships': ['http://swapi.co/api/starships/12/'],
     'url': 'http://swapi.co/api/people/19/',
     'vehicles': []},
    '_type': 'people'},
   {'_id': '26',
    '_index': 'sw',
    '_score': 2.734601,
    '_source': {'birth_year': '37BBY',
     'created': '2014-12-15T13:01:57.178000Z',
     'edited': '2014-12-20T21:17:50.359000Z',
     'eye_color': 'blue',
     'films': ['http://swapi.co/a

In [129]:
random_dict = {i:k for k,i in enumerate('abcd')}

In [144]:
es.index(index='sw', doc_type='rnd',body={'randomshit':'randomvalue'})

{'_id': 'AVg_Dh_S_B7S2ac22Wvm',
 '_index': 'sw',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'rnd',
 '_version': 1,
 'created': True}

In [168]:
es.index(index='sw', doc_type='whatever',id=999, body={'r100':{'r2':'r3'}, 'bla':['1','2','3'], 'sra':{'a':'b'}})

{'_id': '999',
 '_index': 'sw',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'whatever',
 '_version': 4,
 'created': False}

In [165]:
es.get('sw',999)

{'_id': '999',
 '_index': 'sw',
 '_source': {'r1': {'r2': 'r3'}},
 '_type': 'whatever',
 '_version': 1,
 'found': True}

In [173]:
es.index(index='sw',doc_type='sw_doc', body={'aaa':'toszukam'})

{'_id': 'AVg_Oo6GuPCvVr-LxkuM',
 '_index': 'sw',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'sw_doc',
 '_version': 1,
 'created': True}

In [187]:
res = es.search(index='sw',doc_type='sw_doc',body={'query':{'match':{"aaa":"toszukam"}}})
res

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVg_OeYSuPCvVr-LxkuL',
    '_index': 'sw',
    '_score': 1.6931472,
    '_source': {'aaa': 'toszukam'},
    '_type': 'sw_doc'},
   {'_id': 'AVg_Oo6GuPCvVr-LxkuM',
    '_index': 'sw',
    '_score': 1.6931472,
    '_source': {'aaa': 'toszukam'},
    '_type': 'sw_doc'}],
  'max_score': 1.6931472,
  'total': 2},
 'timed_out': False,
 'took': 2}

In [195]:
es.search(index='sw',body={'query':{'match':{'name':'Darth Maul'}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '4',
    '_index': 'sw',
    '_score': 0.39406553,
    '_source': {'birth_year': '41.9BBY',
     'created': '2014-12-10T15:18:20.704000Z',
     'edited': '2014-12-20T21:17:50.313000Z',
     'eye_color': 'yellow',
     'films': ['http://swapi.co/api/films/6/',
      'http://swapi.co/api/films/3/',
      'http://swapi.co/api/films/2/',
      'http://swapi.co/api/films/1/'],
     'gender': 'male',
     'hair_color': 'none',
     'height': '202',
     'homeworld': 'http://swapi.co/api/planets/1/',
     'mass': '136',
     'name': 'Darth Vader',
     'skin_color': 'white',
     'species': ['http://swapi.co/api/species/1/'],
     'starships': ['http://swapi.co/api/starships/13/'],
     'url': 'http://swapi.co/api/people/4/',
     'vehicles': []},
    '_type': 'people'}],
  'max_score': 0.39406553,
  'total': 1},
 'timed_out': False,
 'took': 5}

In [203]:
es.index(index='test',doc_type='test2', body={'zacoto':"100000.0"})

{'_id': 'AVg_RyNQuPCvVr-LxkuP',
 '_index': 'test',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'test2',
 '_version': 1,
 'created': True}

In [197]:
r = requests.get('http://swapi.co/api/people/1')
    

In [205]:
M = json.loads(r.text)

In [208]:
for k in M:
    try:
        M[k] = float(M[k])
    except:
        pass

In [161]:
import logging

logger = logging.getLogger('lgsth')
logger.addHandler(logstash.TCPLogstashHandler('localhost', port=5959))
logger.setLevel(logging.DEBUG)
logger.debug('bla')
    


In [179]:
import logging
import logstash
import sys

host = '0.0.0.0'

test_logger = logging.getLogger('python-logstash-logger')
test_logger.setLevel(logging.INFO)
logstash.UDPLogstashHandler('0.0.0.0', 5959)
test_logger.addHandler(logstash.UDPLogstashHandler('0.0.0.0', 5959))
# test_logger.addHandler(logstash.TCPLogstashHandler('0.0.0.0', 5959))

test_logger.error('python-logstash: test logstash error message.')
test_logger.info('python-logstash: test logstash info message.')
test_logger.warning('python-logstash: test logstash warning message.')

# add extra field to logstash message
extra = {
    'test_string': 'python version: ' + repr(sys.version_info),
    'test_boolean': True,
    'test_dict': {'a': 1, 'b': 'c'},
    'test_float': 1.23,
    'test_integer': 123,
#     'test_list': [1, 2, '3'],
}
test_logger.info('python-logstash: test extra fields', extra=extra)

In [178]:
es.indices.get('*').keys()

dict_keys(['.kibana', 'logstash-2016.11.08'])

In [155]:
test_logger.info('python-logstash: test extra fields', extra=extra)